In [ ]:
#!git clone https://github.com/feihuzhang/GANet.git

# Code replicated and adjusted from: 
# Zhang, Feihu and Priscariu et al. 2019. 
# GA-Net: Guided Aggregation Net for End-to-end Stereo Matching. 
# Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. p.185–194.

In [ ]:
# # Cheat to get more RAM from Colab
# a = []
# while(1):
#     a.append('1')

In [2]:
# https://medium.com/@nrezaeis/pytorch-in-google-colab-640e5d166f13
# Cuda version check
# !nvcc --version

# Python version check
# !python --version

# Check  GPU setup
import torch
torch.cuda.get_device_name(0)



'Tesla P100-PCIE-16GB'

In [2]:
!/opt/bin/nvidia-smi

Tue Jun 30 18:14:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
import os
import torch
# If run from on a separate drive directory
os.chdir('/content/drive/My Drive/Computer_Vision_edit/GANet-master')
# Print pytorch path
# print(os.path.dirname(torch.__file__))

In [ ]:
# !git clone https://github.com/NVIDIA/apex
# Install apex, required for executing the code

%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master/apex 
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./


In [ ]:
# Setup, run this code before executing the rest of the code
# compile.sh
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
%cd libs/GANet
!python setup.py clean
%rm -rf build
!python setup.py build
%cp -r build/lib* build/lib

%cd ../sync_bn
!python setup.py clean
%rm -rf build
!python setup.py build
%cp -r build/lib* build/lib


In [ ]:
# train.sh

%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
# resolution crop; 240h x 528w
!python train.py --crop_height 48 \
                 --crop_width 96 \
                 #--resume '/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_96x48_b16_' \
                 --nEpochs 100 \
                 --batchSize 16 \
                 --kitti2015 1 \
                 --data_path '/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/training/' \
                 --save_path '/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x96_BS16/kitti2015_96x48_b16'


In [ ]:
# Install OpCounter to compute MACs using Lyken17's OpCounter library
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master/
!pip install --upgrade git+https://github.com/Lyken17/pytorch-OpCounter.git

In [ ]:
# Test MACs and parameters counter below
# from torchvision.models import resnet50
# from thop import profile
# model = resnet50()
# input = torch.randn(1, 3, 224, 224)
# macs, params = profile(model, inputs=(input, ))
# print("MACS: {}, PARAMS: {}".format(macs,params))

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.Bottleneck'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.ResNet'>. Treat it as zero Macs and zero Params.
MACS: 4111514624.0, PARAMS: 25557032.0


In [ ]:
# Count the number of MACs and parameters
from thop import profile
from models.GANet_26 import GANet


model = GANet(192)
# nr of parameters
total_num = sum(p.numel() for p in model.parameters())
print(total_num)
# # nr of MACs
# input = torch.randn(1, 3, 48, 48) # torch.randn([16, 3, 48, 144]) # x: [16, 3, 48, 144], y: [16, 3, 48, 144]
# macs, params = profile(model, inputs=(input,input, ))
# print("MACS: {}, PARAMS: {}".format(macs,params))

In [ ]:
# Train with validation
# To measure the number of MACs, set the cuda to 0 and adjust the last lines (144-) in the GA-Net code, located at GANet-master/libs/GANet/modules/GANet.py
# To use another model, change model to the desired version of the GA-Net (GA-Net 26, for example, uses 26 3D Convolutions)
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python train_learning_curve.py --crop_height 48 \
                 --crop_width 144 \
                 --nEpochs 590 \
                 --batchSize 16 \
                 --kitti2015 1 \
                 --data_path '/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/training/' \
                 --training_list 'lists/kitti2015_train_4of5.list' \
                 --val_list 'lists/kitti2015_validate_1of5.list' \
                 --save_path '/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16_3D22/kitti2015_48x144_BS16_3D22_v4' \
                 # --cuda 0 \
                 #--resume '/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16_3D22/kitti2015_48x144_BS16_3D22_v3_epoch_50.pth' \
                 #--model 'GANet_26'\
                 #--lr 0.01


In [ ]:
# Evaluate the model of  the validation set
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python evaluation.py --crop_height=384 \
                  --crop_width=1248 \
                  --max_disp=192 \
                  --data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/training/' \
                  --test_list='lists/kitti2015_validate_1of5.list' \
                  --save_path='./result_eval/48x144_BS16_3D22/' \
                  --resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16_3D22/kitti2015_48x144_BS16_3D22_v3_epoch_640.pth' \
                  --threshold=3.0 \
                  --kitti2015=1 \
                  #--saveimages =1 

In [ ]:
# Prune evaluations with varying sparsities
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python evaluation_prune.py --crop_height=384 \
                  --crop_width=1248 \
                  --max_disp=192 \
                  --data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/training/' \
                  --test_list='lists/kitti2015_validate_1of5.list' \
                  --save_path='./result_eval/Prune/base' \
                  --resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint_prune/kitti2015_48x144_b16_epoch_400.pth' \
                  --threshold=3.0 \
                  --kitti2015=1 \
                  #--saveimages =1 

/content/drive/My Drive/Computer_Vision_edit/GANet-master
Namespace(crop_height=384, crop_width=1248, cuda=True, data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/training/', kitti=0, kitti2015=1, max_disp=192, multi_gpu=0, resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint_prune/kitti2015_48x144_b16_epoch_400.pth', save_path='./result_eval/Prune/base', test_list='lists/kitti2015_validate_1of5.list', threshold=3.0)
===> Building model
=> loading checkpoint '/content/drive/My Drive/Computer_Vision_edit/checkpoint_prune/kitti2015_48x144_b16_epoch_400.pth'
===> Setting Sparsity to 0.0.
===> Frame 0: 000160_10.png ==> EPE Error: 0.9227, Error Rate: 0.0373
===> Frame 1: 000009_10.png ==> EPE Error: 0.9956, Error Rate: 0.0223
===> Frame 2: 000137_10.png ==> EPE Error: 1.0472, Error Rate: 0.0560
===> Frame 3: 000087_10.png ==> EPE Error: 1.1427, Error Rate: 0.0409
===> Frame 4: 000150_10.png ==> EPE Error: 1.8762, Error Rate: 0.0636
===> Frame 5: 00

In [23]:
# Evaluate the results on the test set

%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python predict.py --crop_height=384 \
                --crop_width=1248 \
                --max_disp=192 \
                --data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/' \
                --test_list='lists/kitti2015_test_only.list' \
                --save_path='./result_test/48x144_BS16_3D22/Epoch_500/' \
                --kitti2015=1 \
                --resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16/kitti2015_48x144_b16_epoch_500.pth' \
                --saveimages =True 
                   

/content/drive/My Drive/Computer_Vision_edit/GANet-master
Namespace(crop_height=384, crop_width=1248, cuda=True, data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/', kitti=0, kitti2015=1, max_disp=192, model='GANet_deep', resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16/kitti2015_48x144_b16_epoch_500.pth', save_path='./result_test/48x144_BS16_3D22/Epoch_500/', saveimages=True, test_list='lists/kitti2015_test_only.list', threshold=3.0)
===> Building model
=> loading checkpoint '/content/drive/My Drive/Computer_Vision_edit/checkpoint_48x144_BS16/kitti2015_48x144_b16_epoch_500.pth'
Number of files: 2
Image saved to ./result_test/48x144_BS16_3D22/Epoch_500/000018_10.png
Image: 0
===> Frame 0: 000018_10.png ==> EPE Error: 22.9083, Error Rate: 0.2403
Image saved to ./result_test/48x144_BS16_3D22/Epoch_500/000042_10.png
Image: 1
===> Frame 1: 000042_10.png ==> EPE Error: 12.5460, Error Rate: 0.1758
===> Total 2 Frames ==> AVG E

In [9]:
# Prune predictions with varying sparsities
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python predict_prune.py --crop_height=384 \
                  --crop_width=1248 \
                  --max_disp=192 \
                  --data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/' \
                  --test_list='lists/kitti2015_test_shortest.list' \
                  --save_path='./result_test/Prune/base/' \
                  --kitti2015=1 \
                  --resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth'\
                  #--saveimages=True 

/content/drive/My Drive/Computer_Vision_edit/GANet-master
Namespace(crop_height=384, crop_width=1248, cuda=True, data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/', kitti=0, kitti2015=1, max_disp=192, model='GANet_deep', resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth', save_path='./result_test/Prune/base/', saveimages=False, test_list='lists/kitti2015_test_shortest.list', threshold=3.0)
===> Building model
=> loading checkpoint '/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth'
===> Setting Sparsity to 0.0.
Number of files: 41
Image: 0
===> Frame 0: 000018_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 1
===> Frame 1: 000036_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 2
===> Frame 2: 000042_10.png ==> EPE Error: 0.0019, Error Rate: 0.0000
Image: 3
===> Frame 3: 000065_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 4
===> Frame 4: 000073_10.png ==> EPE Er

In [ ]:
# The trained network from gitHub
%cd /content/drive/My Drive/Computer_Vision_edit/GANet-master
!python predict.py --crop_height=384 \
                  --crop_width=1248 \
                  --max_disp=192 \
                  --data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/'  \
                  --test_list='lists/kitti2015_test.list' \
                  --save_path='./result_original/testset/' \
                  --kitti2015=1 \
                  --resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth' \
                  #--saveimages =True 

/content/drive/My Drive/Computer_Vision_edit/GANet-master
Namespace(crop_height=384, crop_width=1248, cuda=True, data_path='/content/drive/My Drive/Computer_Vision_edit/Datasets/data_scene_flow/testing/', kitti=0, kitti2015=1, max_disp=192, model='GANet_deep', resume='/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth', save_path='./result_original/testset/', saveimages=False, test_list='lists/kitti2015_test.list', threshold=3.0)
===> Building model
=> loading checkpoint '/content/drive/My Drive/Computer_Vision_edit/checkpoint/kitti2015_final.pth'
Number of files: 200
Image: 0
===> Frame 0: 000000_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 1
===> Frame 1: 000001_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 2
===> Frame 2: 000002_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 3
===> Frame 3: 000003_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image: 4
===> Frame 4: 000004_10.png ==> EPE Error: 0.0020, Error Rate: 0.0000
Image